In [1]:
#Lendo a data de hoje no sistema
from datetime import date
today = date. today()    

#Listando arquivos presentes no diretório
from os import listdir
from os.path import isfile, join
path = 'tables/'

files = [f for f in listdir(path) if isfile(join(path, f))]
if(len(files)==0):
    print('[FRX] There is no files in directory. =(')
else:
    #buscando o arquivo do dia
    try:
        filepath = [s for s in files if 'fii_'+str(today) in s][0]
    except:
        print('[FRX] There is no table today. =(')
        filepath = ''

    if(len(filepath)>0):
        import pandas as pd
        df = pd.read_csv(path+filepath)
        
        #  0 - link                     -
        #  1 - Código do fundo          -
        #  2 - Setor                    -
        #  3 - Preço Atual              -
        #  4 - Liquidez Diária          -
        #  5 - Dividendo                -
        #  6 - Dividend Yield           -
        #  7 - DY (3M) Acumulado        -
        #  8 - DY (6M) Acumulado        -
        #  9 - DY (12M) Acumulado
        # 10 - DY (3M) Média
        # 11 - DY (6M) Média
        # 12 - DY (12M) Média
        # 13 - DY Ano
        # 14 - Variação Preço
        # 15 - Rentab. Período
        # 16 - Rentab. Acumulada
        # 17 - Patrimônio Líq.
        # 18 - VPA
        # 19 - P/VPA
        # 20 - DY Patrimonial
        # 21 - Variação Patrimonial
        # 22 - Rentab. Patr. no Período
        # 23 - Rentab. Patr. Acumulada
        # 24 - Vacância Física
        # 25 - Vacância Financeira
        # 26 - Quantidade Ativos

        #Categorias
        df['Código do fundo'] = df['Código do fundo'].astype('category')
        df['link'] = df['link'].astype('category')
        df['Setor'] = df['Setor'].astype('category')
        
        #int32
        df['Quantidade Ativos'] = df['Quantidade Ativos'].astype('int32')
        
        #float64
        float_mask = [4,19]
        for f in float_mask:
            #df[df.columns[f]] = pd.to_numeric(df[df.columns[f]].str.replacece(',','.',regex=False), errors='coerce')
            df[df.columns[f]] = df[df.columns[f]].apply(lambda x: float(str(x).replace(',','.')))
        
        #moedas
        df['Preço Atual'] = pd.to_numeric(df['Preço Atual'].str.replace('.','', regex=False).str.replace("R$ ","", regex=False).str.replace(',','.',regex=False),errors='coerce')
        df['Dividendo'] = pd.to_numeric(df['Dividendo'].str.replace('.','', regex=False).str.replace("R$ ","", regex=False).str.replace(',','.',regex=False),errors='coerce')
        df['Patrimônio Líq.'] = pd.to_numeric(df['Patrimônio Líq.'].str.replace('.','', regex=False).str.replace("R$ ","", regex=False).str.replace(',','.',regex=False),errors='coerce')
        df['VPA'] = pd.to_numeric(df['VPA'].str.replace('.','', regex=False).str.replace("R$ ","", regex=False).str.replace(',','.',regex=False),errors='coerce')
        
        
        #porcentagens
        percent_mask = [6,7,8,9,10,11,12,13,14,15,16,20,21,22,23,24,25]
        
        for p in percent_mask:
            df[df.columns[p]] = pd.to_numeric(df[df.columns[p]].str.replace("%","",regex=False).str.replace(',','.',regex=False),errors='coerce')


In [2]:
#Distância P/VPA = 1

df['Dif 1'] = abs(df['P/VPA']-1)


#Score
df['Score'] = (
    (df['Dividend Yield ']-df['Dividend Yield '].min())/(df['Dividend Yield '].max()-df['Dividend Yield '].min()))*0.3+(
    (df['DY (12M) Acumulado']-df['DY (12M) Acumulado'].min())/(df['DY (12M) Acumulado'].max()-df['DY (12M) Acumulado'].min()))*0.2+(
    (df['DY (12M) Média']-df['DY (12M) Média'].min())/(df['DY (12M) Média'].max()-df['DY (12M) Média'].min()))*0.15+(
    (df['Rentab. Acumulada']-df['Rentab. Acumulada'].min())/(df['Rentab. Acumulada'].max()-df['Rentab. Acumulada'].min()))*0.1+(
    (df['Liquidez Diária']-df['Liquidez Diária'].min())/(df['Liquidez Diária'].max()-df['Liquidez Diária'].min()))*0.1+(
    1-((df['Dif 1']-df['Dif 1'].min())/(df['Dif 1'].max()-df['Dif 1'].min())))*0.1+(
    (df['Quantidade Ativos']-df['Quantidade Ativos'].min())/(df['Quantidade Ativos'].max()-df['Quantidade Ativos'].min()))*0.05





df.sort_values(by='Score', ascending=False, na_position='last').head(10)

,link,Código do fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,...,P/VPA,DY Patrimonial,Variação Patrimonial,Rentab. Patr. no Período,Rentab. Patr. Acumulada,Vacância Física,Vacância Financeira,Quantidade Ativos,Dif 1,Score
60,/funds/rbds11,RBDS11,Residencial,170.23,252.0,6.74,3.00,5.49,9.38,22.10,...,1.48,5.88,-17.13,-12.26,-32.52,NaN,NaN,0,0.48,0.705309
59,/funds/pord11,PORD11,Títulos e Val. Mob.,107.50,248.0,4.35,3.89,3.89,3.89,10.56,...,1.04,NaN,NaN,NaN,NaN,NaN,NaN,0,0.04,0.625465
35,/funds/euro11,EURO11,Logística,173.00,315.0,6.85,3.98,4.73,5.81,8.64,...,0.54,0.19,-0.01,0.18,1.12,31.85,NaN,6,0.46,0.585022
21,/funds/famb11b,FAMB11B,Lajes Corporativas,2985.00,88.0,28.98,0.97,2.86,6.46,15.00,...,0.92,0.92,0.40,1.32,7.88,0.00,NaN,1,0.08,0.489102
17,/funds/tfof11,TFOF11,Títulos e Val. Mob.,143.49,201.0,1.27,0.88,4.36,8.06,14.28,...,1.13,0.74,0.53,1.28,7.09,NaN,NaN,0,0.13,0.456640
2,/funds/bbfi11b,BBFI11B,Lajes Corporativas,2620.01,148.0,25.71,0.95,2.72,5.79,12.98,...,0.85,0.79,-0.01,0.78,6.04,38.59,NaN,2,0.15,0.442481
56,/funds/mfii11,MFII11,Híbrido,116.95,4791.0,1.08,0.91,2.76,5.84,11.79,...,1.17,1.13,-1.20,-0.08,7.74,NaN,NaN,9,0.17,0.430596
20,/funds/brcr11,BRCR11,Híbrido,92.82,55218.0,0.41,0.44,1.25,2.88,14.72,...,0.84,0.37,0.25,0.62,-1.36,15.30,9.6,11,0.16,0.415309
72,/funds/xpcm11,XPCM11,Lajes Corporativas,58.30,8486.0,0.85,1.46,3.71,6.36,11.66,...,0.70,1.02,0.11,1.13,7.00,0.00,0.0,1,0.30,0.410366
97,/funds/ubsr11,UBSR11,Títulos e Val. Mob.,110.00,19960.0,0.86,0.78,2.40,4.64,10.95,...,1.14,0.92,0.04,0.96,8.36,NaN,NaN,0,0.14,0.390143


In [3]:
df.describe()

,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,DY (3M) Média,DY (6M) Média,DY (12M) Média,...,P/VPA,DY Patrimonial,Variação Patrimonial,Rentab. Patr. no Período,Rentab. Patr. Acumulada,Vacância Física,Vacância Financeira,Quantidade Ativos,Dif 1,Score
count,142.000000,1.420000e+02,143.000000,143.000000,141.000000,141.000000,141.000000,140.000000,140.000000,140.000000,...,142.000000,129.000000,129.000000,129.000000,128.000000,78.000000,12.000000,143.000000,142.000000,139.000000
mean,297.346408,1.782185e+04,1.925035,0.590769,1.609716,3.082624,6.146241,0.539643,0.517286,0.515500,...,1.087958,0.661085,-0.106279,0.547442,7.258750,12.886282,15.793333,3.230769,0.201197,0.299126
std,567.807041,8.634845e+04,4.331315,0.509810,0.869823,1.679496,3.921290,0.288734,0.278009,0.325272,...,0.313743,0.514976,1.829979,1.554970,37.840483,20.052038,27.669630,7.773821,0.255849,0.088462
min,1.300000,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.100000,0.020000,-17.130000,-12.260000,-32.520000,0.000000,0.000000,0.000000,0.000000,0.088651
25%,93.250000,2.027500e+02,0.445000,0.425000,1.150000,2.540000,3.960000,0.380000,0.427500,0.337500,...,0.982500,0.490000,-0.060000,0.390000,2.387500,0.000000,1.140000,0.000000,0.050000,0.248627
50%,111.310000,1.538500e+03,0.640000,0.550000,1.670000,3.350000,6.940000,0.560000,0.560000,0.580000,...,1.060000,0.610000,0.010000,0.650000,4.150000,5.395000,8.290000,1.000000,0.130000,0.308816
75%,156.050000,1.226925e+04,0.895000,0.635000,2.010000,4.000000,8.260000,0.670000,0.670000,0.695000,...,1.207500,0.720000,0.190000,0.940000,6.910000,15.172500,12.202500,2.000000,0.297500,0.338832
max,3085.000000,1.014290e+06,28.980000,3.980000,5.490000,9.380000,22.100000,1.830000,1.560000,1.840000,...,3.050000,5.880000,4.870000,5.490000,425.770000,100.000000,100.000000,64.000000,2.050000,0.705309


In [4]:
#Lista Desesejos
wish_list =['HGRE11','GGRC11','VILG11','XPML11']

carteira_michelle = ['MGFF11','CBOP11','XPIN11','MALL11','RBRF11','FLMA11','KNRI11','VISC11','MXRF11']
carteira_wesley = ['MGFF11','CBOP11','XPIN11','MALL11','RBRF11','FLMA11','HGBS11','HGLG11']

#Lista Negra
black_list = ['XPCM11','PORD11','FAMB11B','RBCB11','TFOF11','BBFI11B','CXCE11B','NVIF11B','CPTS11B','VTLT11','RBVO11','BCIA11','CEOC11','KNIP11','RBBV11','MBRF11']
black_list_setor = ['Hospital','Residencial']

#Intervalo aceitável do P/VPA
pvpa_m = df['P/VPA'].median()
pvpa_des = abs(1-pvpa_m)
pvpa_max = 1 + pvpa_des
pvpa_min = 1 - pvpa_des

#liquidez Diária mínima
ld_min = df['Liquidez Diária'].quantile(0.25)

#DY com tesouro selic
selic_am = 0.54
selic_aa = 6.5

#Vacância mínima (%)
vfisica_max = 10
vfinanc_max = 10

In [5]:
#Filtro
#dff = df[df['P/VPA']<=pvpa_max][df['P/VPA']>=pvpa_min][df['Liquidez Diária']>ld_min][df['Dividend Yield ']>selic_am][df['DY (12M) Acumulado']>selic_aa][~df['Código do fundo'].isin(black_list)][~df['Setor'].isin(black_list_setor)][~(df['Vacância Física']>=10)][~(df['Vacância Financeira']>=10)].sort_values(by='Score', ascending=False, na_position='last')
dff = df[df['Dif 1']<=df['Dif 1'].quantile(0.25)][df['Liquidez Diária']>ld_min][df['Dividend Yield ']>selic_am][df['DY (12M) Acumulado']>selic_aa][~df['Código do fundo'].isin(black_list)][~df['Setor'].isin(black_list_setor)][~(df['Vacância Física']>=10)][~(df['Vacância Financeira']>=10)].sort_values(by='Score', ascending=False, na_position='last')

/home/wesley/.pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
#dff['Preço Compra MAX'] = dff['VPA']*pvpa_max
#dff['Preço Compra MIN'] = dff['VPA']*pvpa_min

dff['Preço Compra MAX'] = dff['VPA']+dff['VPA']*df['Dif 1'].quantile(0.25)
dff['Preço Compra MIN'] = dff['VPA']-dff['VPA']*df['Dif 1'].quantile(0.5)

dff['Desejado'] = dff['Código do fundo'].isin(wish_list)

In [7]:
filtro_colunas = ['link',
                  'DY (3M) Acumulado',
                  'DY (6M) Acumulado',
                  'DY (3M) Média',
                  'DY (6M) Média',
                  'DY Ano',
                  'Patrimônio Líq.',
                  'DY Patrimonial',
                  'Variação Patrimonial',
                  'Rentab. Patr. no Período',
                  'Rentab. Patr. Acumulada',
                  'Variação Preço',
                  'Dif 1']

dff = dff.drop(filtro_colunas, axis =1)

# TOP FIIs

In [8]:
dff

,Código do fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (12M) Acumulado,DY (12M) Média,Rentab. Período,Rentab. Acumulada,VPA,P/VPA,Vacância Física,Vacância Financeira,Quantidade Ativos,Score,Preço Compra MAX,Preço Compra MIN,Desejado
102,HFOF11,Títulos e Val. Mob.,104.29,17328.0,0.58,0.55,10.00,0.83,-2.61,16.80,100.02,1.04,NaN,NaN,0,0.366480,105.0210,87.0174,False
90,OUJP11,Títulos e Val. Mob.,106.59,8947.0,0.63,0.59,9.90,0.82,-8.11,7.50,104.83,1.02,NaN,NaN,0,0.361137,110.0715,91.2021,False
104,XPLG11,Logística,103.00,46023.0,0.61,0.57,7.75,0.65,1.08,10.97,98.52,1.05,0.0,NaN,6,0.335726,103.4460,85.7124,False
105,OUCY11,Títulos e Val. Mob.,100.00,4034.0,0.64,0.63,7.44,0.62,4.21,18.47,99.16,1.01,NaN,NaN,0,0.333594,104.1180,86.2692,False
95,BCFF11,Títulos e Val. Mob.,93.40,35221.0,0.57,0.61,7.50,0.62,-4.40,15.97,88.57,1.05,NaN,NaN,0,0.331929,92.9985,77.0559,False
16,FOFT11,Títulos e Val. Mob.,95.71,2956.0,0.56,0.59,7.99,0.67,0.05,6.26,100.64,0.95,NaN,NaN,0,0.328665,105.6720,87.5568,False
116,XPIN11,Outros,115.75,26218.0,0.72,0.61,7.14,0.59,1.66,2.94,113.66,1.02,3.3,NaN,3,0.319637,119.3430,98.8842,False
51,JSRE11,Híbrido,110.71,21747.0,0.62,0.56,6.52,0.54,-0.29,13.02,111.04,1.00,1.0,NaN,3,0.314071,116.5920,96.6048,False


# Lista de desejos

In [9]:
dfwl = df[df['Código do fundo'].isin(wish_list)].drop(filtro_colunas, axis = 1).sort_values(by='Score', ascending=False, na_position='last')
dfwl

,Código do fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (12M) Acumulado,DY (12M) Média,Rentab. Período,Rentab. Acumulada,VPA,P/VPA,Vacância Física,Vacância Financeira,Quantidade Ativos,Score
94,XPML11,Shoppings,112.68,75309.0,0.57,0.52,6.75,0.56,-0.31,8.28,103.43,1.09,2.70,NaN,10,0.317668
89,GGRC11,Logística,140.61,12054.0,0.71,0.51,6.81,0.57,-1.94,15.75,117.75,1.19,0.00,0.00,15,0.316516
14,HGRE11,Lajes Corporativas,155.00,19409.0,0.74,0.49,5.88,0.49,-8.73,7.75,152.12,1.02,18.94,18.81,20,0.307156
129,VILG11,Logística,120.70,19997.0,0.71,0.59,0.00,0.00,1.69,27.88,105.75,1.14,0.00,NaN,3,0.217148


# Carteiras

## Michelle

In [10]:
df[df['Código do fundo'].isin(carteira_michelle)].drop(filtro_colunas, axis = 1).sort_values(by='Score', ascending=False, na_position='last')

,Código do fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (12M) Acumulado,DY (12M) Média,Rentab. Período,Rentab. Acumulada,VPA,P/VPA,Vacância Física,Vacância Financeira,Quantidade Ativos,Score
100,MGFF11,Títulos e Val. Mob.,101.50,41273.0,0.70,0.69,9.26,0.77,3.34,15.13,95.70,1.06,NaN,NaN,0,0.365611
92,RBRF11,Títulos e Val. Mob.,107.70,17468.0,0.70,0.67,8.56,0.71,0.90,7.10,99.17,1.09,NaN,NaN,0,0.343207
93,VISC11,Shoppings,113.00,47850.0,0.92,0.81,6.95,0.58,2.22,11.52,100.40,1.13,5.60,NaN,10,0.340674
10,CBOP11,Lajes Corporativas,83.75,844.0,0.52,0.63,7.90,0.66,1.90,18.39,78.88,1.06,6.87,6.91,1,0.338987
55,MXRF11,Híbrido,10.89,96997.0,0.07,0.64,7.31,0.61,-8.28,1.67,9.82,1.11,NaN,NaN,0,0.324384
116,XPIN11,Outros,115.75,26218.0,0.72,0.61,7.14,0.59,1.66,2.94,113.66,1.02,3.30,NaN,3,0.319637
96,MALL11,Shoppings,102.90,18635.0,0.61,0.59,7.14,0.59,-4.94,0.80,109.63,0.94,0.00,NaN,1,0.312306
53,KNRI11,Híbrido,156.40,45970.0,0.74,0.47,5.99,0.50,3.55,9.46,150.68,1.04,3.01,6.98,18,0.308788
33,FLMA11,Híbrido,4.15,46040.0,0.01,0.33,5.56,0.46,-1.09,47.20,2.89,1.43,5.49,NaN,1,0.286324


## Wesley

In [11]:
df[df['Código do fundo'].isin(carteira_wesley)].drop(filtro_colunas, axis = 1).sort_values(by='Score', ascending=False, na_position='last')

,Código do fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (12M) Acumulado,DY (12M) Média,Rentab. Período,Rentab. Acumulada,VPA,P/VPA,Vacância Física,Vacância Financeira,Quantidade Ativos,Score
100,MGFF11,Títulos e Val. Mob.,101.50,41273.0,0.70,0.69,9.26,0.77,3.34,15.13,95.70,1.06,NaN,NaN,0,0.365611
92,RBRF11,Títulos e Val. Mob.,107.70,17468.0,0.70,0.67,8.56,0.71,0.90,7.10,99.17,1.09,NaN,NaN,0,0.343207
10,CBOP11,Lajes Corporativas,83.75,844.0,0.52,0.63,7.90,0.66,1.90,18.39,78.88,1.06,6.87,6.91,1,0.338987
116,XPIN11,Outros,115.75,26218.0,0.72,0.61,7.14,0.59,1.66,2.94,113.66,1.02,3.30,NaN,3,0.319637
11,HGBS11,Shoppings,254.45,14439.0,1.40,0.55,7.02,0.59,2.19,12.01,216.46,1.18,4.79,NaN,10,0.317149
96,MALL11,Shoppings,102.90,18635.0,0.61,0.59,7.14,0.59,-4.94,0.80,109.63,0.94,0.00,NaN,1,0.312306
13,HGLG11,Logística,154.60,26599.0,0.75,0.49,6.15,0.51,-3.96,9.17,127.14,1.22,6.50,9.70,14,0.298532
33,FLMA11,Híbrido,4.15,46040.0,0.01,0.33,5.56,0.46,-1.09,47.20,2.89,1.43,5.49,NaN,1,0.286324


# Simulador Wesley

In [12]:
#Simulador WESLEY
setores = ['Títulos e Val. Mob.','Outros','Lajes Corporativas','Híbrido','Shoppings','Logística']

#Dados manuais
saldo_disponivel = 3943.94
carteira = [3323.52, 234.20, 891.22, 33.68, 2656.90,2566.15]

#Estratégia de carteira
p_carteira = [0.25, 0.05, 0.20, 0.10, 0.20, 0.20]

dcw = pd.DataFrame({'Setor': setores, '%': p_carteira, 'Investido': carteira})
dcw['Distribuição Ideal'] = round((saldo_disponivel+dcw['Investido'].sum())*dcw['%'],2)
#dcw['Compra Recomendada'] = dcw['Distribuição Ideal'].apply(lambda x: x-dcw['Investido'] if x-dcw['Investido'] > 0 else 0)
dcw['Comprar'] = dcw['Distribuição Ideal']-dcw['Investido'] >= 0

if dcw[dcw['Distribuição Ideal']-dcw['Investido'] < 0].shape[0]>0:
    dcw.loc[(dcw['Distribuição Ideal']-dcw['Investido'] >= 0), 'Disponível para compra'] = round(saldo_disponivel*(dcw['%']/dcw[dcw['Comprar'] == True]['%'].sum()),2)
else:
    dcw.loc[(dcw['Distribuição Ideal']-dcw['Investido'] >= 0), 'Disponível para compra'] = dcw['Distribuição Ideal']-dcw['Investido']
dcw.loc[(dcw['Distribuição Ideal']-dcw['Investido'] < 0), 'Disponível para compra'] = 0.0

dcw.drop('Comprar', axis = 1)

,Setor,%,Investido,Distribuição Ideal,Disponível para compra
0,Títulos e Val. Mob.,0.25,3323.52,3412.40,88.88
1,Outros,0.05,234.20,682.48,448.28
2,Lajes Corporativas,0.20,891.22,2729.92,1838.70
3,Híbrido,0.10,33.68,1364.96,1331.28
4,Shoppings,0.20,2656.90,2729.92,73.02
5,Logística,0.20,2566.15,2729.92,163.77


In [13]:
dcw['Investido'].sum()

9705.67

# Simulador MICHELLE

In [14]:
#Simulador MICHELLE
setores = ['Títulos e Val. Mob.','Outros','Lajes Corporativas','Híbrido','Shoppings','Logística']


#Dados manuais
saldo_disponivel = 6436.76
carteira = [8652.86, 1735.50, 8502.58, 2458.40, 6901.32, 0.0]

#Estratégia de carteira
p_carteira = [0.25, 0.05, 0.20, 0.10, 0.20, 0.20]

dcm= pd.DataFrame({'Setor': setores, '%': p_carteira, 'Investido': carteira})
dcm['Distribuição Ideal'] = round((saldo_disponivel+dcm['Investido'].sum())*dcm['%'],2)
dcm['Comprar'] = dcm['Distribuição Ideal']-dcm['Investido'] >= 0

if dcm[dcm['Distribuição Ideal']-dcm['Investido'] < 0].shape[0]>0:
    dcm.loc[(dcm['Distribuição Ideal']-dcm['Investido'] >= 0), 'Disponível para compra'] = round(saldo_disponivel*(dcm['%']/dcm[dcm['Comprar'] == True]['%'].sum()),2)
else:
    dcm.loc[(dcm['Distribuição Ideal']-dcm['Investido'] >= 0), 'Disponível para compra'] = dcm['Distribuição Ideal']-dcm['Investido']

dcm.loc[(dcm['Distribuição Ideal']-dcm['Investido'] < 0), 'Disponível para compra'] = 0.0

dcm.drop('Comprar', axis = 1)

,Setor,%,Investido,Distribuição Ideal,Disponível para compra
0,Títulos e Val. Mob.,0.25,8652.86,8671.86,2145.59
1,Outros,0.05,1735.50,1734.37,0.00
2,Lajes Corporativas,0.20,8502.58,6937.48,0.00
3,Híbrido,0.10,2458.40,3468.74,858.23
4,Shoppings,0.20,6901.32,6937.48,1716.47
5,Logística,0.20,0.00,6937.48,1716.47


In [15]:
dcm['Investido'].sum()

28250.660000000003

In [16]:
import xlwt
import xlsxwriter

writer = pd.ExcelWriter('tables/top_fii_' + str(today) + '.xls', engine='xlsxwriter')

dff.to_excel(writer,'top_fii','-','%.2f', index=False, encoding='xlwt')
dfwl.to_excel(writer,'lista_desejos','-','%.2f', index=False, encoding='xlwt')
dcm.drop('Comprar', axis = 1).to_excel(writer,'rec_michelle','-','%.2f', index=False, encoding='xlwt')
dcw.drop('Comprar', axis = 1).to_excel(writer,'rec_wesley','-','%.2f', index=False, encoding='xlwt')


writer.save()